In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import sqlalchemy
import folium
from folium import plugins
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import datetime as dt

import os

In [2]:
import folium
from folium.plugins import MarkerCluster
import pysal as ps
from pysal.viz import mapclassify

import ipywidgets as widgets

In [3]:
import partridge as ptg
idx = pd.IndexSlice

In [4]:
def positions_from_db(db_name, limit, offset=0, routes=['all']):
    cols = 'oid, trip_id, route_id, trip_start_time, trip_start_date, vehicle_id, occupancy_status, position_latitude, position_longitude, timestamp'
    aws_host='gtfs-rt-logging.clakglowlpps.us-west-2.rds.amazonaws.com'
    eng = sqlalchemy.create_engine(
        f'mysql://halfempty:tentoninety@{aws_host}:3306/{db_name}')
    
    ##timestamp in db is UTC, dict contains tz offsets for each agency
    ##brittle, rewrite better or change for DST...
    tz_utc_minus = {'mbta': 4, 'emery_go_round': 7, 'mmvta': 4,
                   'vctc_intercity': 7, 'pullman_transit': 7,
                   'rock_reg_metro': 5, 'shreveport': 5}
    
    where = ''
    sep = "'"
    
    if offset == 0:
        offset = ''
    else:
        offset = f'offset {offset}'
    if routes[0] != 'all':
        where = f"where route_id in {str(tuple(routes))} and weekday(timestamp) in (0,1,2,3,4)"
    else:
        where = 'where weekday(timestamp) in (0,1,2,3,4)'

    query = f'select {cols} from vehicle_positions {where} order by timestamp desc limit {limit} {offset}'
    print(query)
    df = pd.read_sql(query, con=eng)
    count = eng.execute('select count(*) from vehicle_positions').fetchall()[0][0]
    
    #apply tz correction
    df['timestamp'] = df['timestamp'] - dt.timedelta(hours=tz_utc_minus[db_name])
    return df, count


In [5]:
def gdf_from_positions_df(df):
    
    df = df.dropna(subset=['position_latitude', 'position_longitude'])
    geo = gpd.points_from_xy(df['position_longitude'], df['position_latitude'])
    gdf = gpd.GeoDataFrame(df, geometry=geo, crs='EPSG:4326')
    
    return gdf

In [6]:
def feed_from_path(path):
    '''Using Partridge, read a (GeoPandas enabled) GTFS feed given a filepath'''
    _date, service_ids = ptg.read_busiest_date(path)
    
    view = {
        'trips.txt': {'service_id': service_ids},
    }
    return ptg.load_geo_feed(path, view)

In [7]:
def aggregate_by_stop(df, gtfs):
    
    routes = df['route_id'].unique()
    
    trips_crowding_rts = gtfs.trips[gtfs.trips['route_id'].isin(routes)]['trip_id']
    stops_crowding_routes = gtfs.stop_times[gtfs.stop_times['trip_id'].isin(trips_crowding_rts)].drop_duplicates(subset=['stop_id'])['stop_id']
    stops_with_crowding = gtfs.stops[gtfs.stops['stop_id'].isin(stops_crowding_routes)]
    stops_with_crowding = stops_with_crowding[['stop_id', 'stop_name', 'geometry']]
    
    if stops_with_crowding.shape[0] == 0:
        rt_id_name = gtfs.routes[['route_id', 'route_short_name']].set_index('route_id')
        trips = gtfs.trips.set_index('route_id').join(rt_id_name)
        trips_crowding_rts = trips[trips['route_short_name'].isin(routes)]['trip_id']
        stops_crowding_routes = gtfs.stop_times[gtfs.stop_times['trip_id'].isin(trips_crowding_rts)].drop_duplicates(subset=['stop_id'])['stop_id']
        stops_with_crowding = gtfs.stops[gtfs.stops['stop_id'].isin(stops_crowding_routes)]
        stops_with_crowding = stops_with_crowding[['stop_id', 'stop_name', 'geometry']]
    
    #https://spatialreference.org/ref/epsg/3586/
    stops_projected = stops_with_crowding.to_crs('EPSG:3586')

    #250ft buffer
    stops_projected['geometry'] = stops_projected['geometry'].buffer(250)
#     return stops_projected

    df = df.to_crs('EPSG:3586')
    ##filter df smaller...
    try:
        df.drop(columns=['position_latitude',
                          'position_longitude'], inplace=True)
    except KeyError:
        pass
    
    display(stops_projected.head(5))
    display(df.head(5))

    join1 = gpd.sjoin(stops_projected, df, how='left', op='contains')
#     return join1
    join1['hour'] = join1['timestamp'].dt.hour
    grouped = join1.groupby(['stop_id', 'route_id', 'hour'])
    series = grouped['occupancy_status'].value_counts().transpose()
    testdf = (pd.DataFrame(series)
              .rename(columns={'occupancy_status':'reports'})
              .unstack(level=[3]))

    ##This is it!
    testdf = testdf.droplevel(0, axis=1)
    geo_df = stops_projected.set_index('stop_id')
    
    return testdf, geo_df

In [8]:
def route_id_from_gtfs(df, gtfs):
    trips = gtfs.trips[['trip_id', 'route_id']].set_index('route_id')
    rt_id_name = gtfs.routes[['route_id', 'route_short_name']].set_index('route_id')
    merge1 = (df.set_index('trip_id')
                .join(trips.join(rt_id_name)
                .set_index('trip_id'))
                 .dropna(subset=['route_short_name']))
    merge1['route_id'] = merge1['route_short_name']
    return merge1

In [9]:
def agg_positions_with_crowding(db_name, gtfs):
    print('Agg runs!')
    #fetch a subset and sense which routes provide crowding data
    subset_allrt, row_count = positions_from_db(db_name, 100000)
#     return subset_allrt

    if (subset_allrt['route_id'] == '').all():
        subset_allrt = route_id_from_gtfs(subset_allrt, gtfs)
        route_data = False
    else:
        route_data = True
        
    display(subset_allrt.head(5))
#     print(row_count)
#     return
    #routes where crowding info isn't all the same, i.e. actual data
    values_by_rt = subset_allrt.groupby('route_id')['occupancy_status'].nunique().eq(1)
    print('values--->',values_by_rt)
    routes_with_crowding = values_by_rt[values_by_rt == False].index
    print('rts_w_crowd--->',routes_with_crowding)
    print(len(routes_with_crowding))
    if list(routes_with_crowding) == ['']:
        print('No actual crowding data!')
        return
    
    ##TODO too small for MBTA but need memory for gtfs stops, implement batching every 10**6 on final.
    i = 0
    while i < row_count:
        if route_data:
            gdf_crowding, _count = positions_from_db(db_name, 5*10**5, 
                                         offset=i, routes=routes_with_crowding)
        else:
            gdf_crowding, _count = positions_from_db(db_name, 5*10**5, 
                                         offset=i,)
            gdf_crowding = route_id_from_gtfs(gdf_crowding, gtfs)
#             return gdf_crowding, routes_with_crowding
            gdf_crowding = gdf_crowding[gdf_crowding['route_id'].isin(routes_with_crowding)]
#             return gdf_crowding
        if gdf_crowding.shape[0] == 0:
            break
        gdf_crowding = gdf_from_positions_df(gdf_crowding)
        aggregated_df, geo_df = aggregate_by_stop(gdf_crowding, gtfs)
        if i == 0:
            global db1
            db1 = aggregated_df
            aggregated_dfs = aggregated_df
        else:
            global db2
            db2 = aggregated_df
            aggregated_dfs = aggregated_dfs.add(aggregated_df, fill_value=0)
        i += 5*10**5
    return aggregated_dfs, geo_df


In [10]:
# gold, gold_geo = (agg_positions_with_crowding('moorpark_city_transit', feed_from_path('./data/to_process/moorpark_city_transit/gtfs.zip')))

In [11]:
def in_stop_list(x):
    try:
        test_list = list(stops_per_rt.loc[x['route_id'].iloc[0]])
    except:
        print(f"no stop data for route {x['route_id'].iloc[0]}!")
        return
    return x[x['stop_id'].isin(test_list)]

In [12]:
def filter_actual_stops(df, gtfs):
    
    global _debug0
    _debug0 = df
    
    def in_stop_list(x):
        try:
            test_list = list(stops_per_rt.loc[x['route_id'].iloc[0]])
        except:
            print(f"no stop data for route {x['route_id'].iloc[0]}!")
            return
        return x[x['stop_id'].isin(test_list)]
    
    display(df.head(5))
    display(gtfs.trips.head(5))
    df_routes = df.index.get_level_values('route_id')
    if not df_routes.isin(gtfs.trips['route_id']).all():
        print('**** new style')
        gtfs_routes = gtfs.routes[['route_id', 'route_short_name']].set_index('route_id')
        trips = gtfs.trips.set_index('route_id').join(gtfs_routes)
        trips = trips.reset_index(drop=True)
        
        global _trips0
        _trips0 = trips
        
        trips = trips.rename(columns={'route_short_name':'route_id'})
        global _trips1
        _trips1 = trips
        
        trip_indexed = trips.set_index('trip_id')[['route_id', 'direction_id']]
        stops_per_rt = gtfs.stop_times.set_index('trip_id').join(trip_indexed).groupby('route_id')['stop_id'].unique()
    else:
        trip_indexed = gtfs.trips.set_index('trip_id')[['route_id', 'direction_id']]
        stops_per_rt = gtfs.stop_times.set_index('trip_id').join(trip_indexed).groupby('route_id')['stop_id'].unique()

        #     return stops_per_rt
    global _debug1
    _debug1 = df
    
    reset = df.reset_index()
    rt_group = reset.groupby('route_id')
    
    print('stops_per_rt--->', stops_per_rt)
    df = rt_group.apply(in_stop_list)
    global _debug2
    _debug2 = df
#     return df
    df = df.set_index('stop_id', append=True).set_index('hour', append=True).droplevel(1)
    
    return df

In [70]:
# mbta_filtered.to_parquet('./data/mbta/mbta_filtered.parquet')

In [71]:
##approx Aug 17 to Sep 4... 
##TODO implement timestamping in df fetch, or from S3 files...

In [12]:
# mbta_filtered = pd.read_parquet('./data/mbta/mbta_filtered.parquet')

In [75]:
# mbta_geo.to_file('./data/mbta/mbta_geo.geojson', driver="GeoJSON")

In [14]:
# mbta_geo = gpd.read_file('./data/mbta/mbta_geo.geojson').set_index('stop_id', drop=True)

In [13]:
def feeds_from_files(folder_path):
    '''
    '''
    feeds = {}
    subdirs = [x[0] for x in os.walk(folder_path)]
    for subdir in subdirs[1:]:
        agency = subdir.split('/')[-1]
        print(agency)
#         continue
#         feeds[agency] = {}
        for feed in os.listdir(subdir):
            if feed[0] == '.':
                continue
            if feed[-4:] != '.zip':
                continue
#             datestr = feed.split('_')[0]
#             date = dt.datetime.strptime(datestr,'%Y%m%d')
#             feeds[agency] = feed_from_path(subdir+'/'+feed)
            gtfs = feed_from_path(subdir+'/'+feed)
#         try:
        agency_df, agency_geo = agg_positions_with_crowding(agency, gtfs)
        agency_filtered = filter_actual_stops(agency_df, gtfs)
        agency_filtered.to_parquet(f'{subdir}/{agency}_filtered.parquet')
        agency_geo.to_file(f'{subdir}/{agency}_geo.geojson', driver="GeoJSON")
#         except:
#             print(f'No apparent crowding data for {agency}')
    return 

In [36]:
feeds_from_files('./data/processed/')

rock_reg_metro
Agg runs!
select oid, trip_id, route_id, trip_start_time, trip_start_date, vehicle_id, occupancy_status, position_latitude, position_longitude, timestamp from vehicle_positions where weekday(timestamp) in (0,1,2,3,4) order by timestamp desc limit 100000 


,oid,route_id,trip_start_time,trip_start_date,vehicle_id,occupancy_status,position_latitude,position_longitude,timestamp,route_short_name
trip_id,,,,,,,,,,
1051-Weekday,307599,2,15:24:00,20200918,3395,STANDING_ROOM_ONLY,34.7216,-92.2816,2020-09-18 15:38:28,2
1051-Weekday,307568,2,15:24:00,20200918,3395,STANDING_ROOM_ONLY,34.7217,-92.2853,2020-09-18 15:37:42,2
1051-Weekday,307537,2,15:24:00,20200918,3395,STANDING_ROOM_ONLY,34.7217,-92.2862,2020-09-18 15:36:55,2
1051-Weekday,307506,2,15:24:00,20200918,3395,STANDING_ROOM_ONLY,34.7249,-92.2861,2020-09-18 15:36:09,2
1051-Weekday,307475,2,15:24:00,20200918,3395,STANDING_ROOM_ONLY,34.7269,-92.2847,2020-09-18 15:35:23,2


values---> route_id
10    False
11    False
13    False
14    False
16    False
18    False
2     False
22    False
23    False
3     False
4     False
5     False
6     False
8     False
Name: occupancy_status, dtype: bool
rts_w_crowd---> Index(['10', '11', '13', '14', '16', '18', '2', '22', '23', '3', '4', '5', '6',
       '8'],
      dtype='object', name='route_id')
14
select oid, trip_id, route_id, trip_start_time, trip_start_date, vehicle_id, occupancy_status, position_latitude, position_longitude, timestamp from vehicle_positions where weekday(timestamp) in (0,1,2,3,4) order by timestamp desc limit 500000 


/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,stop_id,stop_name,geometry
0,1506992,Pulaski Tech South,"POLYGON ((-5630515.210 913929.759, -5630516.41..."
1,1506997,13328 Frontage Rd,"POLYGON ((-5632052.940 913610.556, -5632054.14..."
2,1506998,13325 Frontage Rd,"POLYGON ((-5632432.691 913291.802, -5632433.89..."
3,1520662,13001 Frontage Rd,"POLYGON ((-5627800.249 913998.481, -5627801.45..."
4,1506999,12819 Frontage Rd,"POLYGON ((-5627492.375 913986.838, -5627493.57..."


...

,stop_id,stop_name,geometry
0,426264,MMVTA Bus Yard Departing,"POLYGON ((-1675457.509 2276282.351, -1675458.7..."
1,344310,Meldon @ 11th,"POLYGON ((-1675749.324 2277906.834, -1675750.5..."
2,321520,Meldon @ 9th (southbound),"POLYGON ((-1675707.892 2276793.351, -1675709.0..."
3,344311,Meldon @ 7th (southbound),"POLYGON ((-1675857.837 2275850.673, -1675859.0..."
4,344312,Meldon @ 6th (soutbound),"POLYGON ((-1675943.787 2275327.670, -1675944.9..."


,oid,route_id,trip_start_time,trip_start_date,vehicle_id,occupancy_status,timestamp,route_short_name,geometry
trip_id,,,,,,,,,
1,104771,ComA,04:25:00,20200918,772,EMPTY,2020-09-18 06:13:40,ComA,POINT (-1705222.315 2375600.037)
1,104762,ComA,04:25:00,20200918,772,EMPTY,2020-09-18 06:12:54,ComA,POINT (-1705589.767 2375563.497)
1,104753,ComA,04:25:00,20200918,772,EMPTY,2020-09-18 06:12:08,ComA,POINT (-1706282.721 2374790.295)
1,104744,ComA,04:25:00,20200918,772,EMPTY,2020-09-18 06:11:22,ComA,POINT (-1706739.535 2374140.014)
1,104736,ComA,04:25:00,20200918,772,EMPTY,2020-09-18 06:10:36,ComA,POINT (-1707039.687 2373657.220)


occupancy_status       EMPTY  MANY_SEATS_AVAILABLE  STANDING_ROOM_ONLY
stop_id route_id hour                                                 
1321285 L2       7.0     5.0                   NaN                 NaN
                 9.0     4.0                   NaN                 NaN
                 11.0    4.0                   1.0                 NaN
                 12.0    1.0                   2.0                 NaN
                 13.0    4.0                   1.0                 NaN

,trip_id,route_id,service_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible
0,1,1212,0,To Pittsburgh,NaN,0,224045,1671,NaN
1,2,1212,0,To Pittsburgh,NaN,0,224046,1671,NaN
2,3,1212,0,To Pittsburgh,NaN,0,224047,1671,NaN
3,4,1212,0,To Pittsburgh,NaN,0,224048,1671,NaN
4,5,1212,0,To Pittsburgh,NaN,0,224049,1671,NaN


**** new style
stops_per_rt---> route_id
Cal C      [492534, 344829, 5241287, 344831, 344832, 3448...
ComA       [426264, 344310, 321520, 344311, 344312, 34431...
E3         [426264, 5238962, 4827668, 345120, 344636, 344...
Ex1        [426264, 344271, 344273, 344282, 344640, 34428...
Ex2        [426264, 344318, 344334, 345120, 344584, 34463...
L2         [446114, 344909, 345142, 344915, 344916, 34491...
L3         [345010, 5221260, 344555, 5988779, 5235765, 52...
Local 1    [345010, 5221260, 344555, 5221421, 5221422, 34...
Name: stop_id, dtype: object


In [ ]:
## clean shit up, try running, target 2-3 to debug...

### next: 
   * view logic, calculate metric
   * visualizer
